In [1]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [2]:
#mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
!pip install --upgrade pyarrow

     |████████████████████████████████| 17.7MB 205kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [5]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Uninstalling tensorflow-2.3.0:
  Successfully uninstalled tensorflow-2.3.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-obuth7i3
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-obuth7i3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 20.9MB/s 
     |████████████████████████████████| 2.9MB 42.7MB/s 
  Created wheel for transformers: filename=transformers-4.0.0rc1-cp36-none-any.whl size=1349233 sha256=0ec7c06134a0a8291270a5afcb0c47d5c850b7919d09e180738cfaf4ab877052
  Stored in directory: /tmp/pip-ephem-wheel-cache-ayhzim6q/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=60d85af3c5a2d3a9bea5b1f0d7908c4989fcbca3a0e72b11a3bfd6a05f6a46d0
  Stored in 

In [6]:
# Check that we have a GPU
!nvidia-smi

Thu Nov 19 19:33:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import BertConfig

config = BertConfig(
    vocab_size=32_000,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
    max_seq_length=200,
)

In [ ]:
from transformers import BertForMaskedLM

model = BertForMaskedLM.from_pretrained('bert-base-cased')# (config=config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.num_parameters()

108340804

In [ ]:
!pwd

/content


In [8]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased", truncation = True, padding=True, max_length=200)

In [9]:
cd "/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work"

/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work


In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./tenk_join_8k.txt",
    block_size=100,
)

CPU times: user 3min 14s, sys: 6.64 s, total: 3min 20s
Wall time: 2min 12s


In [ ]:
!pwd

/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
!pwd

/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./tenkBERTv2Cased",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:281: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [ ]:
#del [trainer]
#gc.collect()
#torch.cuda.empty_cache()

In [ ]:
%%time
trainer.train()

Step,Training Loss
500,4.971075
1000,4.559712
1500,4.400961
2000,4.233240
2500,4.089010
3000,4.071043
3500,3.967018
4000,3.899508
4500,3.863074
5000,3.765777


CPU times: user 4h 42min 56s, sys: 3h 26min 30s, total: 8h 9min 27s
Wall time: 8h 11min 38s


TrainOutput(global_step=48558, training_loss=3.1042982747436056)

In [ ]:
trainer.save_model("./tenkBERTv2Cased")

In [10]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./tenkBERTv2Cased",
    tokenizer = tokenizer
)

Some weights of BertModel were not initialized from the model checkpoint at ./tenkBERTv2Cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
fill_mask("Securities [MASK] Section Act registered.")

[{'score': 0.9264759421348572,
  'sequence': '[CLS] Securities Exchange Section Act registered. [SEP]',
  'token': 7855,
  'token_str': 'Exchange'},
 {'score': 0.020196685567498207,
  'sequence': '[CLS] Securities means Section Act registered. [SEP]',
  'token': 2086,
  'token_str': 'means'},
 {'score': 0.014569743536412716,
  'sequence': '[CLS] Securities applicable Section Act registered. [SEP]',
  'token': 13036,
  'token_str': 'applicable'},
 {'score': 0.010680689476430416,
  'sequence': '[CLS] Securities retained Section Act registered. [SEP]',
  'token': 5366,
  'token_str': 'retained'},
 {'score': 0.00211513158865273,
  'sequence': '[CLS] Securities Rule Section Act registered. [SEP]',
  'token': 11575,
  'token_str': 'Rule'}]

In [14]:
fill_mask("income [MASK] cash equity stockholders consolidated flows")

[{'score': 0.36285004019737244,
  'sequence': '[CLS] income statement cash equity stockholders consolidated flows [SEP]',
  'token': 4195,
  'token_str': 'statement'},
 {'score': 0.28655579686164856,
  'sequence': '[CLS] income statements cash equity stockholders consolidated flows [SEP]',
  'token': 8477,
  'token_str': 'statements'},
 {'score': 0.13705137372016907,
  'sequence': '[CLS] income comprehensive cash equity stockholders consolidated flows [SEP]',
  'token': 8232,
  'token_str': 'comprehensive'},
 {'score': 0.04039079323410988,
  'sequence': '[CLS] income reported cash equity stockholders consolidated flows [SEP]',
  'token': 2103,
  'token_str': 'reported'},
 {'score': 0.023556502535939217,
  'sequence': '[CLS] income Company cash equity stockholders consolidated flows [SEP]',
  'token': 1881,
  'token_str': 'Company'}]

In [15]:
fill_mask("Reports [MASK] Statements 8-K  Form.")

[{'score': 0.9265838265419006,
  'sequence': '[CLS] Reports Financial Statements 8 - K Form. [SEP]',
  'token': 7748,
  'token_str': 'Financial'},
 {'score': 0.010293220169842243,
  'sequence': '[CLS] Reports Current Statements 8 - K Form. [SEP]',
  'token': 9493,
  'token_str': 'Current'},
 {'score': 0.008259247988462448,
  'sequence': '[CLS] Reports Consolidated Statements 8 - K Form. [SEP]',
  'token': 23582,
  'token_str': 'Consolidated'},
 {'score': 0.005152004770934582,
  'sequence': '[CLS] Reports Index Statements 8 - K Form. [SEP]',
  'token': 10146,
  'token_str': 'Index'},
 {'score': 0.0046477061696350574,
  'sequence': '[CLS] Reports Annual Statements 8 - K Form. [SEP]',
  'token': 8451,
  'token_str': 'Annual'}]

In [16]:
fill_mask("hereto [MASK] signing parties execute may WITNESS hereto Amendment parties caused WHEREOF")

[{'score': 0.18073439598083496,
  'sequence': '[CLS] hereto party signing parties execute may WITNESS hereto Amendment parties caused WHEREOF [SEP]',
  'token': 1710,
  'token_str': 'party'},
 {'score': 0.08155486732721329,
  'sequence': '[CLS] hereto Amendment signing parties execute may WITNESS hereto Amendment parties caused WHEREOF [SEP]',
  'token': 10060,
  'token_str': 'Amendment'},
 {'score': 0.06686510145664215,
  'sequence': '[CLS] hereto counterparts signing parties execute may WITNESS hereto Amendment parties caused WHEREOF [SEP]',
  'token': 15289,
  'token_str': 'counterparts'},
 {'score': 0.058904729783535004,
  'sequence': '[CLS] hereto deliver signing parties execute may WITNESS hereto Amendment parties caused WHEREOF [SEP]',
  'token': 7852,
  'token_str': 'deliver'},
 {'score': 0.03446617349982262,
  'sequence': '[CLS] hereto document signing parties execute may WITNESS hereto Amendment parties caused WHEREOF [SEP]',
  'token': 5830,
  'token_str': 'document'}]

In [19]:
fill_mask("AAR [MASK] Hardware Corp")

[{'score': 0.07756713032722473,
  'sequence': '[CLS] AAR International Hardware Corp [SEP]',
  'token': 1570,
  'token_str': 'International'},
 {'score': 0.06534997373819351,
  'sequence': '[CLS] AAR Technology Hardware Corp [SEP]',
  'token': 3529,
  'token_str': 'Technology'},
 {'score': 0.053845006972551346,
  'sequence': '[CLS] AAR Systems Hardware Corp [SEP]',
  'token': 6475,
  'token_str': 'Systems'},
 {'score': 0.026818500831723213,
  'sequence': '[CLS] AAR Delaware Hardware Corp [SEP]',
  'token': 8056,
  'token_str': 'Delaware'},
 {'score': 0.026250671595335007,
  'sequence': '[CLS] AAR USA Hardware Corp [SEP]',
  'token': 3066,
  'token_str': 'USA'}]

In [24]:
fill_mask("Directors Officers Registrant [MASK]")

[{'score': 0.9742537140846252,
  'sequence': '[CLS] Directors Officers Registrant Executive [SEP]',
  'token': 4183,
  'token_str': 'Executive'},
 {'score': 0.006062740925699472,
  'sequence': '[CLS] Directors Officers Registrant Corporation [SEP]',
  'token': 3436,
  'token_str': 'Corporation'},
 {'score': 0.0026572123169898987,
  'sequence': '[CLS] Directors Officers Registrant since [SEP]',
  'token': 1290,
  'token_str': 'since'},
 {'score': 0.00252664927393198,
  'sequence': '[CLS] Directors Officers Registrant The [SEP]',
  'token': 1109,
  'token_str': 'The'},
 {'score': 0.0013238827232271433,
  'sequence': '[CLS] Directors Officers Registrant indicated [SEP]',
  'token': 4668,
  'token_str': 'indicated'}]

In [28]:
fill_mask("contracts losses results operations exchange [MASK] included")

[{'score': 0.5227145552635193,
  'sequence': '[CLS] contracts losses results operations exchange currency included [SEP]',
  'token': 10202,
  'token_str': 'currency'},
 {'score': 0.07192683964967728,
  'sequence': '[CLS] contracts losses results operations exchange Foreign included [SEP]',
  'token': 4201,
  'token_str': 'Foreign'},
 {'score': 0.05204450711607933,
  'sequence': '[CLS] contracts losses results operations exchange rates included [SEP]',
  'token': 5600,
  'token_str': 'rates'},
 {'score': 0.04037196934223175,
  'sequence': '[CLS] contracts losses results operations exchange rate included [SEP]',
  'token': 2603,
  'token_str': 'rate'},
 {'score': 0.04010125622153282,
  'sequence': '[CLS] contracts losses results operations exchange gains included [SEP]',
  'token': 12535,
  'token_str': 'gains'}]

In [31]:
fill_mask("insurance [MASK] issued instruments conjunction These typically")

[{'score': 0.17400027811527252,
  'sequence': '[CLS] insurance contracts issued instruments conjunction These typically [SEP]',
  'token': 8216,
  'token_str': 'contracts'},
 {'score': 0.05478797107934952,
  'sequence': '[CLS] insurance subsidiaries issued instruments conjunction These typically [SEP]',
  'token': 22423,
  'token_str': 'subsidiaries'},
 {'score': 0.04620804637670517,
  'sequence': '[CLS] insurance issued issued instruments conjunction These typically [SEP]',
  'token': 3010,
  'token_str': 'issued'},
 {'score': 0.04371991381049156,
  'sequence': '[CLS] insurance purchase issued instruments conjunction These typically [SEP]',
  'token': 4779,
  'token_str': 'purchase'},
 {'score': 0.032342325896024704,
  'sequence': '[CLS] insurance payments issued instruments conjunction These typically [SEP]',
  'token': 10772,
  'token_str': 'payments'}]

In [32]:
fill_mask("lower [MASK] Inventories market valued")

[{'score': 0.6482276916503906,
  'sequence': '[CLS] lower cost Inventories market valued [SEP]',
  'token': 2616,
  'token_str': 'cost'},
 {'score': 0.049574825912714005,
  'sequence': '[CLS] lower costs Inventories market valued [SEP]',
  'token': 4692,
  'token_str': 'costs'},
 {'score': 0.03291061148047447,
  'sequence': '[CLS] lower generally Inventories market valued [SEP]',
  'token': 2412,
  'token_str': 'generally'},
 {'score': 0.03241129219532013,
  'sequence': '[CLS] lower stated Inventories market valued [SEP]',
  'token': 2202,
  'token_str': 'stated'},
 {'score': 0.019756190478801727,
  'sequence': '[CLS] lower fair Inventories market valued [SEP]',
  'token': 4652,
  'token_str': 'fair'}]

In [33]:
fill_mask("Item [MASK] accounts exposure discussion")

[{'score': 0.1275644451379776,
  'sequence': '[CLS] Item Risk accounts exposure discussion [SEP]',
  'token': 19547,
  'token_str': 'Risk'},
 {'score': 0.09983118623495102,
  'sequence': '[CLS] Item risk accounts exposure discussion [SEP]',
  'token': 3187,
  'token_str': 'risk'},
 {'score': 0.08705148100852966,
  'sequence': '[CLS] Item see accounts exposure discussion [SEP]',
  'token': 1267,
  'token_str': 'see'},
 {'score': 0.07111045718193054,
  'sequence': '[CLS] Item additional accounts exposure discussion [SEP]',
  'token': 2509,
  'token_str': 'additional'},
 {'score': 0.03487299755215645,
  'sequence': '[CLS] Item See accounts exposure discussion [SEP]',
  'token': 3969,
  'token_str': 'See'}]

In [34]:
fill_mask("KPMG [MASK] Company")

[{'score': 0.09487592428922653,
  'sequence': '[CLS] KPMG acquired Company [SEP]',
  'token': 2888,
  'token_str': 'acquired'},
 {'score': 0.09403404593467712,
  'sequence': '[CLS] KPMG Holding Company [SEP]',
  'token': 14382,
  'token_str': 'Holding'},
 {'score': 0.0526903010904789,
  'sequence': '[CLS] KPMG subsidiaries Company [SEP]',
  'token': 22423,
  'token_str': 'subsidiaries'},
 {'score': 0.044131964445114136,
  'sequence': '[CLS] KPMG purchase Company [SEP]',
  'token': 4779,
  'token_str': 'purchase'},
 {'score': 0.040667030960321426,
  'sequence': '[CLS] KPMG The Company [SEP]',
  'token': 1109,
  'token_str': 'The'}]

In [37]:
fill_mask("INFORMATION [MASK] hereunder CONFIDENTIAL rights")

[{'score': 0.584905743598938,
  'sequence': '[CLS] INFORMATION obligations hereunder CONFIDENTIAL rights [SEP]',
  'token': 15337,
  'token_str': 'obligations'},
 {'score': 0.035476814955472946,
  'sequence': '[CLS] INFORMATION privileges hereunder CONFIDENTIAL rights [SEP]',
  'token': 15676,
  'token_str': 'privileges'},
 {'score': 0.03140512853860855,
  'sequence': '[CLS] INFORMATION exercise hereunder CONFIDENTIAL rights [SEP]',
  'token': 6730,
  'token_str': 'exercise'},
 {'score': 0.028166629374027252,
  'sequence': '[CLS] INFORMATION interests hereunder CONFIDENTIAL rights [SEP]',
  'token': 4740,
  'token_str': 'interests'},
 {'score': 0.025194982066750526,
  'sequence': '[CLS] INFORMATION except hereunder CONFIDENTIAL rights [SEP]',
  'token': 2589,
  'token_str': 'except'}]